### Algorithm to Approximate Stabilizer Rank
* Input: Linear combination of stabilizer states
$$\ket{\Psi} = \sum_{j=1}^N c_j\ket{\psi_j}$$
* Output: Following Pair
    * (i) List of canonical stabilizer matrices
    $$M = \{\mathcal{M}_1, \mathcal{M}_2, ..., \mathcal{M}_N\}$$
    * (ii) List of coefficients 
    $$C = \{\mathcal{c}_1, \mathcal{c}_2, ..., \mathcal{c}_N\}$$
* Procedure: https://arxiv.org/abs/1711.07848 - On the Geometry of Stabilizer States - Section 5.3